<a href="https://colab.research.google.com/github/tmrb/Master-project-codes/blob/main/Preprocessingdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import sys
drive.mount("/content/drive", force_remount=True)
root  = '/content/drive/My Drive/projeto'
sys.path.append(root)

Mounted at /content/drive


In [ ]:
!pip install -U scikit-learn>=0.20
!pip install feature_selector

     |████████████████████████████████| 716kB 21.8MB/s 
     |████████████████████████████████| 184kB 59.0MB/s 
     |████████████████████████████████| 12.2MB 39.4MB/s 
     |████████████████████████████████| 36.2MB 135kB/s 
     |████████████████████████████████| 13.1MB 311kB/s 
     |████████████████████████████████| 9.5MB 181kB/s 
  Created wheel for seaborn: filename=seaborn-0.8.1-cp37-none-any.whl size=184835 sha256=baf811cf23d6885b5cb62878faadd1bf84fe3212236248de957b050c694a9087
  Stored in directory: /root/.cache/pip/wheels/26/0a/44/53ddd89769e62f7c6691976375b86c6492e7dd20a2d3970e32
  Created wheel for matplotlib: filename=matplotlib-2.1.2-cp37-cp37m-linux_x86_64.whl size=10244001 sha256=9142c9e433d01685848e899de109fc12ed2ed9b0e69f1245ba11e116bb47f28b
  Stored in directory: /root/.cache/pip/wheels/83/9d/e6/cd76b09328833d683b447bc658dbd436bc9af0cd152c654407
  ERROR: Failed building wheel for pandas
  Running setup.py clean for pandas
  ERROR: Failed building wheel for scikit-lear

In [ ]:
from feature_selector import FeatureSelector
import pandas as pd

ModuleNotFoundError: ignored

In [ ]:
train = pd.read_csv('/content/drive/My Drive/projeto/overall4backup.csv',encoding='ISO-8859-1',sep = ';')
train_labels = train['Situação']
train.head()

In [ ]:
train = train.drop(columns = ['Situação'])

Create the Instance
The FeatureSelector only requires a dataset with observations in the rows and features in the columns (standard structured data). We are working with a classifified machine learning problem so we also pass in training labels.

In [ ]:
fs = FeatureSelector(data = train, labels = train_labels)

In [ ]:
fs.identify_missing(missing_threshold=0.6)


2. Single Unique Value
The next method is straightforward: find any features that have only a single unique value. (This does not one-hot encode the features).

In [ ]:
fs.identify_single_unique()

3. Collinear (highly correlated) Features
This method finds pairs of collinear features based on the Pearson correlation coefficient. For each pair above the specified threshold (in terms of absolute value), it identifies one of the variables to be removed. We need to pass in a correlation_threshold.

This method is based on code found at https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/

For each pair, the feature that will be removed is the one that comes last in terms of the column ordering in the dataframe. (This method does not one-hot encode the data beforehand unless one_hot=True. Therefore correlations are only calculated between numeric columns)

In [ ]:
fs.identify_collinear(correlation_threshold=0.975)

In [ ]:
correlated_features = fs.ops['collinear']
correlated_features[:]

In [ ]:
fs.plot_collinear()

In [ ]:
fs.plot_collinear(plot_all=True)

In [ ]:
fs.identify_collinear(correlation_threshold=0.98)
fs.plot_collinear()

In [ ]:
fs.record_collinear.head()

4. Zero Importance Features
This method relies on a machine learning model to identify features to remove. It therefore requires a supervised learning problem with labels. The method works by finding feature importances using a gradient boosting machine implemented in the LightGBM library.

To reduce variance in the calculated feature importances, the model is trained a default 10 times. The model is also by default trained with early stopping using a validation set (15% of the training data) to identify the optimal number of estimators to train. The following parameters can be passed to the identify_zero_importance method:

task: either classification or regression. The metric and labels must match with the task
eval_metric: the metric used for early stopping (for example auc for classification or l2 for regression). To see a list of available metrics, refer to the LightGBM docs
n_iterations: number of training runs. The feature importances are averaged over the training runs (default = 10)
early_stopping: whether to use early stopping when training the model (default = True). Early stopping stops training estimators (decision trees) when the performance on a validation set no longer decreases for a specified number of estimators (100 by default in this implementation). Early stopping is a form of regularization used to prevent overfitting to training data
The data is first one-hot encoded for use in the model. This means that some of the zero importance features may be created from one-hot encoding. To view the one-hot encoded columns, we can access the one_hot_features of the FeatureSelector.

Note of caution: in contrast to the other methods, the feature imporances from a model are non-deterministic (have a little randomness). The results of running this method can change each time it is run.

In [ ]:
fs.identify_zero_importance(task = 'classification', eval_metric = 'auc', 
                            n_iterations = 10, early_stopping = True)

In [ ]:
one_hot_features = fs.one_hot_features
base_features = fs.base_features
print('There are %d original features' % len(base_features))
print('There are %d one-hot features' % len(one_hot_features))

In [ ]:
zero_importance_features = fs.ops['zero_importance']
zero_importance_features[10:15]

**Plot Feature Importances**
The feature importance plot using plot_feature_importances will show us the plot_n most important features (on a normalized scale where the features sum to 1). It also shows us the cumulative feature importance versus the number of features.

When we plot the feature importances, we can pass in a threshold which identifies the number of features required to reach a specified cumulative feature importance. For example, threshold = 0.99 will tell us the number of features needed to account for 99% of the total importance.

In [ ]:
fs.plot_feature_importances(threshold = 0.99, plot_n = 12)

In [ ]:
fs.feature_importances.head(10)

In [ ]:
one_hundred_features = list(fs.feature_importances.loc[:34, 'feature'])
len(one_hundred_features)

**Low Importance Features**
This method builds off the feature importances from the gradient boosting machine (identify_zero_importance must be run first) by finding the lowest importance features not needed to reach a specified cumulative total feature importance. For example, if we pass in 0.99, this will find the lowest important features that are not needed to reach 99% of the total feature importance.

When using this method, we must have already run identify_zero_importance and need to pass in a cumulative_importance that accounts for that fraction of total feature importance.

Note of caution: this method builds on the gradient boosting model features importances and again is non-deterministic. I advise running these two methods several times with varying parameters and testing each resulting set of features rather than picking one number and sticking to it.

In [ ]:
fs.identify_low_importance(cumulative_importance = 0.99)

In [ ]:
low_importance_features = fs.ops['low_importance']
low_importance_features[:5]

**Removing Features**
Once we have identified the features to remove, we have a number of ways to drop the features. We can access any of the feature lists in the removal_ops dictionary and remove the columns manually. We also can use the remove method, passing in the methods that identified the features we want to remove.

This method returns the resulting data which we can then use for machine learning. The original data will still be accessible in the data attribute of the Feature Selector.

Be careful of the methods used for removing features! It's a good idea to inspect the features that will be removed before using the remove function.

In [ ]:
train_no_missing = fs.remove(methods = ['missing'])

In [ ]:
train_no_missing_zero = fs.remove(methods = ['missing', 'zero_importance'])

In [ ]:
all_to_remove = fs.check_removal()
all_to_remove[10:25]

In [ ]:
train_removed = fs.remove(methods = 'all')

In [ ]:
train_removed.head()

In [ ]:
train_removed_all = fs.remove(methods = 'all', keep_one_hot=False)

In [ ]:
print('Original Number of Features', train.shape[1])
print('Final Number of Features: ', train_removed_all.shape[1])

In [ ]:
!fusermount -u drive
!google-drive-ocamlfuse drive

In [ ]:
# Import Drive API and authenticate.
from google.colab import drive

# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

# Write the DataFrame to CSV file.
with open('/gdrive/My Drive/processedoverall.csv', 'w') as f:
  train_removed_all.to_csv(f)